Doğrusal Olmayan Regresyon Modelleri

In [68]:
# GEREKLİ KÜTÜPHANELER
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import neighbors
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb

In [2]:
# Önemsiz Hatalardan Kurtulmak İçin
from warnings import filterwarnings
filterwarnings("ignore")

1-K En Yakın Komşu (KNN)

In [3]:
# Gözlemlerin birbirine olan benzerlikleri üzerinden tahmin yapılır.

In [4]:
df = pd.read_csv("Hitters.csv")

In [5]:
df.head(10)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A
6,185,37,1,23,8,21,2,214,42,1,30,9,24,N,E,76,127,7,70.0,A
7,298,73,0,24,24,7,3,509,108,0,41,37,12,A,W,121,283,9,100.0,A
8,323,81,6,26,32,8,2,341,86,6,32,34,8,N,W,143,290,19,75.0,N
9,401,92,17,49,66,65,13,5206,1332,253,784,890,866,A,E,0,0,0,1100.0,A


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      322 non-null    int64  
 1   Hits       322 non-null    int64  
 2   HmRun      322 non-null    int64  
 3   Runs       322 non-null    int64  
 4   RBI        322 non-null    int64  
 5   Walks      322 non-null    int64  
 6   Years      322 non-null    int64  
 7   CAtBat     322 non-null    int64  
 8   CHits      322 non-null    int64  
 9   CHmRun     322 non-null    int64  
 10  CRuns      322 non-null    int64  
 11  CRBI       322 non-null    int64  
 12  CWalks     322 non-null    int64  
 13  League     322 non-null    object 
 14  Division   322 non-null    object 
 15  PutOuts    322 non-null    int64  
 16  Assists    322 non-null    int64  
 17  Errors     322 non-null    int64  
 18  Salary     263 non-null    float64
 19  NewLeague  322 non-null    object 
dtypes: float64

In [7]:
df = df.dropna() # Eksik bilgileri sildik

In [8]:
df

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


In [9]:
dms = pd.get_dummies(df[["League","Division","NewLeague"]]) # Kategorik verileri nümerik değerlere çevirdik

In [10]:
dms

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,False,True,False,True,False,True
2,True,False,False,True,True,False
3,False,True,True,False,False,True
4,False,True,True,False,False,True
5,True,False,False,True,True,False
...,...,...,...,...,...,...
317,False,True,True,False,False,True
318,True,False,True,False,True,False
319,True,False,False,True,True,False
320,True,False,True,False,True,False


In [11]:
y = df["Salary"] # Bağımlı değişkeni y'ye atadık

In [12]:
y

1       475.0
2       480.0
3       500.0
4        91.5
5       750.0
        ...  
317     700.0
318     875.0
319     385.0
320     960.0
321    1000.0
Name: Salary, Length: 263, dtype: float64

In [13]:
x_ = df.drop(["Salary","League","Division","NewLeague"], axis=1).astype("float64") # Bağımlı değişkeni ve kategorik değişkenleri attık


In [14]:
x_

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497.0,127.0,7.0,65.0,48.0,37.0,5.0,2703.0,806.0,32.0,379.0,311.0,138.0,325.0,9.0,3.0
318,492.0,136.0,5.0,76.0,50.0,94.0,12.0,5511.0,1511.0,39.0,897.0,451.0,875.0,313.0,381.0,20.0
319,475.0,126.0,3.0,61.0,43.0,52.0,6.0,1700.0,433.0,7.0,217.0,93.0,146.0,37.0,113.0,7.0
320,573.0,144.0,9.0,85.0,60.0,78.0,8.0,3198.0,857.0,97.0,470.0,420.0,332.0,1314.0,131.0,12.0


In [15]:
x = pd.concat([x_, dms[["League_N",'Division_W',"NewLeague_N"]]], axis=1) # bağımsız değişkenlerimizle, bağımlı değişkenleri ve dummiesleri bir araya getirdik.

In [16]:
x

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,True,True,True
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,False,True,False
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,True,False,True
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,True,False,True
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497.0,127.0,7.0,65.0,48.0,37.0,5.0,2703.0,806.0,32.0,379.0,311.0,138.0,325.0,9.0,3.0,True,False,True
318,492.0,136.0,5.0,76.0,50.0,94.0,12.0,5511.0,1511.0,39.0,897.0,451.0,875.0,313.0,381.0,20.0,False,False,False
319,475.0,126.0,3.0,61.0,43.0,52.0,6.0,1700.0,433.0,7.0,217.0,93.0,146.0,37.0,113.0,7.0,False,True,False
320,573.0,144.0,9.0,85.0,60.0,78.0,8.0,3198.0,857.0,97.0,470.0,420.0,332.0,1314.0,131.0,12.0,False,False,False


In [17]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.25, random_state=42)

In [18]:
x_train

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
183,328.0,91.0,12.0,51.0,43.0,33.0,2.0,342.0,94.0,12.0,51.0,44.0,33.0,145.0,59.0,8.0,True,False,True
229,514.0,144.0,0.0,67.0,54.0,79.0,9.0,4739.0,1169.0,13.0,583.0,374.0,528.0,229.0,453.0,15.0,True,False,True
286,593.0,152.0,23.0,69.0,75.0,53.0,6.0,2765.0,686.0,133.0,369.0,384.0,321.0,315.0,10.0,6.0,False,True,False
102,233.0,49.0,2.0,41.0,23.0,18.0,8.0,1350.0,336.0,7.0,166.0,122.0,106.0,102.0,132.0,10.0,False,False,False
153,341.0,95.0,6.0,48.0,42.0,20.0,10.0,2964.0,808.0,81.0,379.0,428.0,221.0,158.0,4.0,5.0,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,629.0,168.0,18.0,73.0,102.0,40.0,18.0,8424.0,2464.0,164.0,1008.0,1072.0,402.0,1067.0,157.0,14.0,False,False,False
236,237.0,52.0,0.0,15.0,25.0,30.0,24.0,14053.0,4256.0,160.0,2165.0,1314.0,1566.0,523.0,43.0,6.0,True,True,True
93,511.0,138.0,25.0,76.0,96.0,61.0,3.0,592.0,164.0,28.0,87.0,110.0,71.0,157.0,7.0,8.0,False,True,False
137,343.0,103.0,6.0,48.0,36.0,40.0,15.0,4338.0,1193.0,70.0,581.0,421.0,325.0,211.0,56.0,13.0,False,False,False


In [19]:
x_test

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
148,497.0,136.0,7.0,58.0,38.0,26.0,11.0,3871.0,1066.0,40.0,450.0,367.0,241.0,304.0,347.0,10.0,False,False,False
154,537.0,147.0,23.0,58.0,88.0,47.0,10.0,2744.0,730.0,97.0,302.0,351.0,174.0,92.0,257.0,20.0,True,False,True
318,492.0,136.0,5.0,76.0,50.0,94.0,12.0,5511.0,1511.0,39.0,897.0,451.0,875.0,313.0,381.0,20.0,False,False,False
279,439.0,96.0,0.0,44.0,36.0,65.0,4.0,711.0,148.0,1.0,68.0,56.0,99.0,229.0,406.0,22.0,True,False,True
88,155.0,41.0,12.0,21.0,29.0,22.0,16.0,5409.0,1338.0,181.0,746.0,805.0,875.0,165.0,9.0,1.0,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,539.0,139.0,5.0,93.0,58.0,69.0,5.0,1469.0,369.0,12.0,247.0,126.0,198.0,462.0,9.0,7.0,False,True,False
282,633.0,210.0,6.0,91.0,56.0,59.0,6.0,3070.0,872.0,19.0,420.0,230.0,274.0,367.0,432.0,16.0,True,True,True
111,325.0,76.0,16.0,33.0,52.0,37.0,5.0,1506.0,351.0,71.0,195.0,219.0,214.0,726.0,87.0,3.0,True,True,False
232,313.0,83.0,9.0,43.0,41.0,30.0,14.0,5885.0,1543.0,104.0,751.0,714.0,535.0,58.0,141.0,23.0,True,True,True


In [20]:
y_train

183     125.000
229    1940.000
286     940.000
102     375.000
153     100.000
         ...   
24      776.667
236     750.000
93      145.000
137     430.000
133      87.500
Name: Salary, Length: 197, dtype: float64

In [ ]:
y_test

1.1-Model

In [ ]:
knnModel = KNeighborsRegressor().fit(x_train, y_train)

In [ ]:
knnModel

In [ ]:
knnModel.n_neighbors

In [ ]:
knnModel.metric

In [ ]:
dir(knnModel)

1.2-Tahmin

In [ ]:
knnModel.predict(x_test)

In [ ]:
yTahmin = knnModel.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test, yTahmin))

In [ ]:
hataKarelerOrtalaması

1.3-Model Tuning

In [ ]:
knnModel

In [ ]:
RMSE = []

for k in range(10):
    k = k + 1
    knnModel = KNeighborsRegressor(n_neighbors=k).fit(x_train,y_train)
    yTahmin = knnModel.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test,yTahmin))
    RMSE.append(rmse)
    print("k=",k ,"için RMSE değeri:", rmse)

In [ ]:
# GridSearchCV Parametre kombinasyonları değerlendirmek ve kıyaslamak için kullanırız.

In [ ]:
knnParams = {"n_neighbors":np.arange(1,30,1)}

In [ ]:
knn = KNeighborsRegressor()

In [ ]:
knnCvModel = GridSearchCV(knn,knnParams,cv=10).fit(x_train,y_train)

In [ ]:
knnCvModel.best_params_

1.4-Final Model

In [ ]:
knnTuned = KNeighborsRegressor(n_neighbors=knnCvModel.best_params_["n_neighbors"]).fit(x_train,y_train)

In [ ]:
knnTuned

In [ ]:
yTahmin = knnTuned.predict(x_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,yTahmin))

2-Destek Vektör Regresyonu(Support Vector Regression)

2.1-Model

In [ ]:
svrModel = SVR(kernel="linear").fit(x_train, y_train)

In [ ]:
svrModel

In [ ]:
svrModel.predict(x_train)

In [ ]:
svrModel.predict(x_test)

In [ ]:
svrModel.intercept_

In [ ]:
svrModel.coef_

2.2-Test

In [ ]:
yTahmin = svrModel.predict(x_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, yTahmin))

2.3-Model Tuning

In [ ]:
svrModel = SVR(kernel="linear")

In [ ]:
svrModel

In [ ]:
svrParams = {"C": [0.1,0.5,1.3]}

In [ ]:
svrCvModel = GridSearchCV(svrModel,svrParams,cv=5).fit(x_train, y_train)

In [ ]:
svrCvModel.best_params_

In [ ]:
svrCvModel = GridSearchCV(svrModel,svrParams,cv=5,verbose = 2,n_jobs= -1).fit(x_train, y_train) #verbose = 2 durumu raporlayarak çalıştırırır, n_jobs=-1 bilgisayar maximum gücünde çalışır

In [ ]:
svrCvModel.best_params_

2.4-Final Modeli

In [ ]:
svrTuned = SVR(kernel="linear",C=0.5).fit(x_train, y_train)

In [ ]:
yTahmin = svrTuned.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

3-Yapay Sinir Ağları

In [ ]:
# Amaç, en küçük hata ile tahmin yapabilecek katsayılara erişmektir

3.1 Model

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler

In [ ]:
scaler.fit(x_train)
x_trainScaled = scaler.transform(x_train)

In [ ]:
x_testScaled = scaler.transform(x_test)

In [ ]:
mlpModel = MLPRegressor().fit(x_trainScaled,y_train)

In [ ]:
mlpModel

3.2-Tahmin

In [ ]:
yTahmin = mlpModel.predict(x_testScaled)

In [ ]:
yTahmin

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

3.3-Model Tuning 

In [ ]:
mlpParams = {
    "alpha": [0.1, 0.01,0.02, 0.001, 0.0001],
    "hidden_layer_sizes": [(10, 10, 10, 10), (20, 20, 20, 20), (100, 100, 100, 100)]}


In [ ]:
mlpCvModel = GridSearchCV(mlpModel,mlpParams,cv=10,verbose=2,n_jobs=-1).fit(x_trainScaled,y_train)

In [ ]:
mlpCvModel.best_params_

3.4-Final Mdel

In [ ]:
mlpTuned = MLPRegressor(alpha=0.01,hidden_layer_sizes=(100,100,100,100)).fit(x_train,y_train)

In [ ]:
mlpTuned.predict(x_test)

In [ ]:
yTahmin = mlpTuned.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

4-CART

4.1-Model

In [ ]:
x_train = pd.DataFrame(x_train["Hits"])

In [ ]:
x_test = pd.DataFrame(x_test["Hits"])

In [ ]:
cartModel = DecisionTreeRegressor(max_leaf_nodes=10)

In [ ]:
cartModel.fit(x_train,y_train)

In [ ]:
xGrid = np.arange(min(np.array(x_train)), max(np.array(x_train)), 0.01)
xGrid = xGrid.reshape((len(xGrid), 1))  # Reshape to a 2D array with a single column

plt.scatter(x_train, y_train, color="red")
plt.plot(xGrid, cartModel.predict(xGrid), color="blue")
plt.title("Cart Regresyon Ağacı")
plt.xlabel("Artış Sayısı(Hits)")
plt.ylabel("Maaş(Salary)")

4.2 Tahmin

In [ ]:
# Tek değişkenli
cartModel.predict(x_test)

In [ ]:
yTahmin = cartModel.predict(x_test) 

In [ ]:
np.sqrt(mean_squared_error(y_test,yTahmin))

4.3 Model Tuning

In [ ]:
cartModel

In [ ]:
cartModel = DecisionTreeRegressor(max_depth=2).fit(x_train,y_train)

In [ ]:
yTahmin = cartModel.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

In [ ]:
cartModel = DecisionTreeRegressor()

In [ ]:
cartParams = {"max_depth":[2,3,4,5,6,7,8,9,10,11,12,13,14],"min_samples_leaf":[2,10,5,30,50,100,200,300,400,500]}

In [ ]:
cartCvModel = GridSearchCV(cartModel,cartParams,cv=10,verbose=2,n_jobs=-1).fit(x_train,y_train)

In [ ]:
cartCvModel

In [ ]:
cartCvModel.best_params_

4.4-Final Modeli

In [ ]:
cartTunedModel = DecisionTreeRegressor(max_depth=10,min_samples_split=50).fit(x_train,y_train)

In [ ]:
cartTunedModel

In [ ]:
yTahmin = cartTunedModel.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

5-Random Forest

In [ ]:
# Temel bootstrap yöntemi ile oluşturulan birden fazla karar ağacının ürettiği tahminlerin bir araya getirilerek değerlendirimesine dayanır.Birden çok karar ağacının ürettiği tahminlerin bir araya getirilerek değerlendirilmesine dayanır.

5.1 Model

In [ ]:
rfModel = RandomForestRegressor(random_state=42).fit(x_train,y_train)

In [ ]:
rfModel

5.2-Tahmin

In [ ]:
yTahmin=rfModel.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

5.3-Model

In [ ]:
rfParams = {"max_depth": range(1,10), "min_samples_split":[1,2,5,10],"n_estimators":[200,500,1000,2000],"min_samples_leaf":[2,10,80,100]}

In [ ]:
rfParams

In [ ]:
rfCvModel = GridSearchCV(rfModel,rfParams, cv=10,n_jobs=-1,verbose=2).fit(x_train,y_train)

In [ ]:
rfCvModel.best_params_

In [ ]:
rfTuned = RandomForestRegressor(random_state=42,
                                max_depth=2,
                                max_features=2,
                                min_samples_leaf=10,
                                n_estimators=1000
                                ).fit(x_train,y_train)

In [ ]:
rfTuned

In [ ]:
yTahmin = rfTuned.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

Değişken Önem Düzeyi

In [ ]:
rfTuned.feature_importances_*100

In [ ]:

Importance = pd.DataFrame({"Importance": rfTuned.feature_importances_ * 100}, index=x_train.columns)
sorted_importance = Importance.sort_values(by="Importance", axis=0, ascending=True)

sorted_importance.plot(kind="barh", color="r")
plt.xlabel("Variable Importance")
plt.legend().set_visible(False)  
plt.show()

6-Gradient Boosting Machines (GBM)

6.1 Model ve Tahmin

In [ ]:
gbmModel = GradientBoostingRegressor().fit(x_train,y_train)

In [ ]:
gbmModel

In [ ]:
yTahmin = gbmModel.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

In [ ]:
gbmParams = {"learning_rate":[0.001,0.1,0.01],
             "max_depth":[3,5,8],
             "n_estimators":[100,200,500],
             "subsample":[1,0.5,0.8],
             "loss":['huber', 'quantile', 'absolute_error', 'squared_error']}

In [ ]:
gbmParams

In [ ]:
gbmModel = GradientBoostingRegressor().fit(x_train,y_train)

In [ ]:
gbmCvModel = GridSearchCV(gbmModel,gbmParams,
                          cv=10,n_jobs=-1,verbose=2).fit(x_train,y_train)

In [ ]:
gbmCvModel.best_params_

In [ ]:
gbmTuned = GradientBoostingRegressor(learning_rate = 0.01,
 loss =  'absolute_error',
 max_depth = 3,
 n_estimators = 200,
 subsample = 0.5).fit(x_train,y_train)

In [ ]:
yTahmin = gbmTuned.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

Değisken Önem Düzeyleri

In [ ]:
Importance = pd.DataFrame({"Importance": gbmTuned.feature_importances_ * 100}, index=x_train.columns)
sorted_importance = Importance.sort_values(by="Importance", axis=0, ascending=True)

sorted_importance.plot(kind="barh", color="r")
plt.xlabel("Variable Importance")
plt.gca().legend_ = None

7-XGBoost

In [ ]:
!pip install xgboost

In [69]:
from xgboost import XGBRegressor

7.1-Model ve Tahmin

In [70]:
xgb = XGBRegressor().fit(x_train,y_train)

In [71]:
xgb

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [72]:
yTahmin = xgb.predict(x_test)

In [73]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [74]:
hataKarelerOrtalaması

366.3863437634965

7.2-Model Tuning

In [75]:
pip install --upgrade joblib xgboost

Note: you may need to restart the kernel to use updated packages.


In [76]:
xgb = XGBRegressor

In [77]:
xgb

xgboost.sklearn.XGBRegressor

In [78]:
xgbParams = {"learing_rate": [0.1,0.01,0.5],
              "max_depth":[2,3,4,5,8],
              "n_estimators":[100,200,500,1000],
              "colsample_bytree": [0.4,0.7,1]}

In [79]:
xgbCvParams = GridSearchCV(xgb, xgbParams, cv=10, n_jobs=1, verbose=2).fit(x_train, y_train)

TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.

In [ ]:
xgbCvParams.best_params_

In [ ]:
xgbTuned = XGBRegressor(colsample_bytree=0.4, 
                        learning_rate=0.1, 
                        max_depth=2, 
                        n_estimators=100).fit(x_train, y_train)

In [ ]:
xgbTuned

In [ ]:
yTahmin = xgbTuned.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(x_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

8-LIGHT GBM

In [ ]:
!pip install lightgbm

In [ ]:
# conda install -c conda-forge lightgbm

8.1-Model ve Tahmin

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
lgmModel = LGBMRegressor().fit(x_train,y_train)

In [ ]:
yTahmin = lgmModel.predict(x_test)

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

8.2-Model Tuning

In [ ]:
lgbmParams = {"learning_rate":[0.01,0.1,0.5,1],
             "max_depth":[1,2,3,4,5,6,7,8,9,10],
             "n_estimators":[20,40,100,200,500,1000]}

In [ ]:
lgbmCvModel = GridSearchCV(lgmModel,lgbmParams,cv=10,n_jobs=-1,verbose=2).fit(x_train,y_train)

In [ ]:
lgbmCvModel.best_params_

In [ ]:
lgmTuned = LGBMRegressor(learning_rate=0.1,
                         max_depth=1,
                         n_estimators=40).fit(x_train,y_train)

In [80]:
yTahmin = lgmTuned.predict(x_test)

NameError: name 'lgmTuned' is not defined

In [ ]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [ ]:
hataKarelerOrtalaması

 8-CatBoost 

In [21]:
!pip install catboost

In [22]:
from catboost import CatBoostRegressor

8.1 Model ve Tahmin

In [23]:
catbModel = CatBoostRegressor().fit(x_train,y_train)

Learning rate set to 0.031674
0:	learn: 437.6430699	total: 157ms	remaining: 2m 37s
1:	learn: 431.3923642	total: 158ms	remaining: 1m 18s
2:	learn: 424.8820360	total: 159ms	remaining: 52.9s
3:	learn: 418.2514904	total: 160ms	remaining: 39.8s
4:	learn: 412.6394021	total: 161ms	remaining: 32s
5:	learn: 406.6247020	total: 161ms	remaining: 26.7s
6:	learn: 400.5321206	total: 162ms	remaining: 23s
7:	learn: 394.6683437	total: 163ms	remaining: 20.2s
8:	learn: 388.2496484	total: 164ms	remaining: 18s
9:	learn: 382.9448842	total: 164ms	remaining: 16.3s
10:	learn: 377.2600080	total: 165ms	remaining: 14.8s
11:	learn: 372.4829606	total: 166ms	remaining: 13.6s
12:	learn: 366.6823437	total: 166ms	remaining: 12.6s
13:	learn: 362.6076230	total: 167ms	remaining: 11.8s
14:	learn: 358.0107745	total: 168ms	remaining: 11s
15:	learn: 353.2802665	total: 168ms	remaining: 10.4s
16:	learn: 348.5646265	total: 169ms	remaining: 9.78s
17:	learn: 343.6407912	total: 170ms	remaining: 9.27s
18:	learn: 339.2363847	total: 17

In [24]:
yTahmin = catbModel.predict(x_test)

In [25]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [26]:
hataKarelerOrtalaması

351.194631344607

8.2 Model Tuning

In [28]:
catbParams = {"iterations":[200,500,1000],
              "learning_rate":[0.01,0.1],
              "depth":[3,6,8]}

In [29]:
catbModel = CatBoostRegressor()

In [33]:
catbCvModel = GridSearchCV(catbModel,catbParams,cv=10,n_jobs=-1,verbose=2).fit(x_train,y_train)

Fitting 10 folds for each of 18 candidates, totalling 180 fits
0:	learn: 425.7900818	total: 402us	remaining: 80.1ms
1:	learn: 404.8723520	total: 930us	remaining: 92.1ms
2:	learn: 387.4057666	total: 1.24ms	remaining: 81.4ms
3:	learn: 372.2801584	total: 1.53ms	remaining: 75.2ms
4:	learn: 358.9204229	total: 1.88ms	remaining: 73.1ms
5:	learn: 347.0083933	total: 2.16ms	remaining: 69.7ms
6:	learn: 336.0130818	total: 2.44ms	remaining: 67.4ms
7:	learn: 324.3923300	total: 2.75ms	remaining: 66ms
8:	learn: 314.8690957	total: 3.05ms	remaining: 64.8ms
9:	learn: 308.5075563	total: 3.38ms	remaining: 64.2ms
10:	learn: 298.8587285	total: 3.65ms	remaining: 62.8ms
11:	learn: 294.7655438	total: 3.94ms	remaining: 61.7ms
12:	learn: 288.0697862	total: 4.25ms	remaining: 61.1ms
13:	learn: 282.6697154	total: 4.53ms	remaining: 60.1ms
14:	learn: 277.6121667	total: 4.83ms	remaining: 59.6ms
15:	learn: 273.4383979	total: 5.1ms	remaining: 58.7ms
16:	learn: 269.1556201	total: 5.36ms	remaining: 57.7ms
17:	learn: 264.80

In [34]:
catbCvModel.best_params_

{'depth': 3, 'iterations': 200, 'learning_rate': 0.1}

In [38]:
catbTuned = CatBoostRegressor(depth=3,iterations=200,learning_rate=0.1).fit(x_train,y_train)

0:	learn: 425.7900818	total: 567us	remaining: 113ms
1:	learn: 404.8723520	total: 1.08ms	remaining: 107ms
2:	learn: 387.4057666	total: 1.41ms	remaining: 92.3ms
3:	learn: 372.2801584	total: 1.7ms	remaining: 83.3ms
4:	learn: 358.9204229	total: 2.02ms	remaining: 78.8ms
5:	learn: 347.0083933	total: 2.31ms	remaining: 74.6ms
6:	learn: 336.0130818	total: 2.64ms	remaining: 72.7ms
7:	learn: 324.3923300	total: 2.93ms	remaining: 70.3ms
8:	learn: 314.8690957	total: 3.26ms	remaining: 69.1ms
9:	learn: 308.5075563	total: 3.58ms	remaining: 68ms
10:	learn: 298.8587285	total: 3.87ms	remaining: 66.5ms
11:	learn: 294.7655438	total: 4.18ms	remaining: 65.5ms
12:	learn: 288.0697862	total: 4.49ms	remaining: 64.5ms
13:	learn: 282.6697154	total: 4.8ms	remaining: 63.8ms
14:	learn: 277.6121667	total: 5.09ms	remaining: 62.8ms
15:	learn: 273.4383979	total: 5.37ms	remaining: 61.8ms
16:	learn: 269.1556201	total: 5.7ms	remaining: 61.4ms
17:	learn: 264.8098704	total: 6.05ms	remaining: 61.2ms
18:	learn: 261.6700768	total

In [39]:
catbTuned

In [41]:
yTahmin = catbTuned.predict(x_test)

In [42]:
yTahmin

array([ 722.46372317,  739.44452032, 1251.01146267,  228.40809895,
        568.1391357 ,  294.82134202,  263.70791045,  110.63536874,
        850.39846471,  477.6679629 ,  875.61311096,  822.59713753,
        629.42167583,  291.1266889 ,  293.05294386, 1100.70110471,
        696.83467334,   82.76119898, 1037.2438691 ,  269.94989996,
        442.30836962,  818.24748001,  795.98255811,  503.81615166,
        487.7809396 ,  100.07299874,  766.41397905,  328.39434047,
        476.43611996,  175.33908422,  152.39258771,  557.14148239,
        513.08901487,  209.62538319,  241.34839213, 1025.82084463,
       1542.24574885,  224.60976128,  222.18456749,  455.08374627,
         77.61516459,  118.26196248,  711.43695161,  610.06173101,
       1273.29340755,  707.28080763,  505.55407192,  116.84586252,
        214.33867342,  684.7285287 ,  634.44605078,  516.4992399 ,
        738.90093826,  162.40811423,  104.53245336,  758.45064869,
        698.41271799,  693.02916774,  243.87570676, 1467.39045

In [43]:
hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test,yTahmin))

In [44]:
hataKarelerOrtalaması

344.3125832615482

9- Makine Öğrenmesi Görevlerinin Otomatikleştirilmesi

In [46]:
df = pd.read_csv("Hitters.csv")
df = df.dropna()  
dms = ms = pd.get_dummies(df[["League","Division","NewLeague"]])

In [90]:
def compML(df,y,alg):
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.preprocessing import scale
    from sklearn.preprocessing import StandardScaler
    from sklearn import model_selection
    from sklearn.linear_model import LinearRegression
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.neural_network import MLPRegressor
    from sklearn import neighbors
    from sklearn.metrics import mean_squared_error, r2_score
    from sklearn.model_selection import train_test_split,GridSearchCV
    from sklearn.ensemble import GradientBoostingRegressor
    from lightgbm import LGBMRegressor
    import xgboost as xgb
    #train-test ayrımı
    y = df[y]
    x_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis=1).astype(
    "float64")  
    x = pd.concat([x_, dms[["League_N", 'Division_W', "NewLeague_N"]]],
              axis=1)  
    x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.25, random_state=42)
    # Modelleme
    model = alg().fit(x_train,y_train)
    yTahmin = model.predict(x_test)
    hataKarelerOrtalaması = np.sqrt(mean_squared_error(y_test, yTahmin))
    modelİsmi = alg.__name__
    print(modelİsmi,"Modeli Test Hatası",hataKarelerOrtalaması)

In [87]:
compML(df,"Salary",SVR)

SVR : 460.0032657244849


In [88]:
models = [LGBMRegressor,
          XGBRegressor,
          GradientBoostingRegressor,
          RandomForestRegressor,
          DecisionTreeRegressor,
          MLPRegressor,
          KNeighborsRegressor,
          SVR]

In [92]:
for i in models:
    compML(df,"Salary",i)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000094 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 831
[LightGBM] [Info] Number of data points in the train set: 197, number of used features: 19
[LightGBM] [Info] Start training from score 543.483442
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 